In [1]:
transaction="E:\\Power_BI\\Customer_Segmentation\\Transaction.csv"
mapping="E:\\Power_BI\\Customer_Segmentation\\rfm_mapping.xlsx"

In [2]:
import pandas as pd
import numpy as np

In [3]:
df1=pd.read_csv(transaction)
df2=pd.read_excel(mapping)

In [4]:
df1.head()

,id,UserId,TransactionId,TransactionTime,ItemCode,ItemDescription,NumberOfItemsPurchased,CostPerItem,Country
0,0,278166,6355745,2019-02-02 12:50:00,465549,FAMILY ALBUM WHITE PICTURE FRAME,6,11.73,United Kingdom
1,1,337701,6283376,2018-12-26 09:06:00,482370,LONDON BUS COFFEE MUG,3,3.52,United Kingdom
2,2,267099,6385599,2019-02-15 09:45:00,490728,SET 12 COLOUR PENCILS DOLLY GIRL,72,0.90,France
3,3,380478,6044973,2018-06-22 07:14:00,459186,UNION JACK FLAG LUGGAGE TAG,3,1.73,United Kingdom
4,4,285957,6307136,2019-01-11 09:50:00,1787247,CUT GLASS T-LIGHT HOLDER OCTAGON,12,3.52,United Kingdom


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380405 entries, 0 to 380404
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      380405 non-null  int64  
 1   UserId                  380405 non-null  int64  
 2   TransactionId           380405 non-null  int64  
 3   TransactionTime         380405 non-null  object 
 4   ItemCode                380405 non-null  int64  
 5   ItemDescription         380405 non-null  object 
 6   NumberOfItemsPurchased  380405 non-null  int64  
 7   CostPerItem             380405 non-null  float64
 8   Country                 380405 non-null  object 
dtypes: float64(1), int64(5), object(3)
memory usage: 26.1+ MB


In [6]:
df1.TransactionTime=pd.to_datetime(df1.TransactionTime)

In [7]:
df2.head()

,r_score,f_score,m_score,RFM_score,Segments
0,3,1,4,314,Big spender
1,3,1,5,315,Big spender
2,3,2,4,324,Big spender
3,3,2,5,325,Big spender
4,3,3,4,334,Big spender


In [8]:
df1['F_value']=df1.TransactionId.count()

In [9]:
rfmtable = df1.groupby('UserId')['TransactionId'].nunique().reset_index(name='F_Value')

In [10]:
from datetime import datetime
today = datetime.today()
rvalue = df1.groupby('UserId')['TransactionTime'].max().apply(lambda x: (today - x).days).reset_index(name='R_Value')

In [11]:
df1['Amount']=df1.CostPerItem*df1.NumberOfItemsPurchased

In [12]:
mvalue = df1.groupby('UserId')['Amount'].sum().reset_index(name='M_Value')

In [13]:
rfmtable = rfmtable.merge(rvalue, on='UserId')

In [14]:
rfmtable = rfmtable.merge(mvalue, on='UserId')

In [15]:
rfmtable

,UserId,F_Value,R_Value,M_Value
0,259266,1,2626,320608.80
1,259287,7,2303,17565.78
2,259308,4,2376,6954.48
3,259329,1,2319,7211.94
4,259350,1,2611,1387.20
...,...,...,...,...
4334,383880,1,2578,748.65
4335,383901,1,2481,334.80
4336,383922,2,2308,739.20
4337,383943,16,2304,8125.65


In [16]:
rfmtable['R_Score'] = pd.qcut(rfmtable['R_Value'], 5, labels=[5,4,3,2,1]).astype(int)
rfmtable['F_Score'] = pd.qcut(rfmtable['F_Value'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
rfmtable['M_Score'] = pd.qcut(rfmtable['M_Value'], 5, labels=[1,2,3,4,5]).astype(int)

In [17]:
rfmtable.head()

,UserId,F_Value,R_Value,M_Value,R_Score,F_Score,M_Score
0,259266,1,2626,320608.80,1,1,5
1,259287,7,2303,17565.78,5,5,5
2,259308,4,2376,6954.48,2,4,4
3,259329,1,2319,7211.94,4,1,4
4,259350,1,2611,1387.20,1,1,2


In [18]:
rfmtable['rfm_score'] = (
    rfmtable['R_Score'].astype(str) +
    rfmtable['F_Score'].astype(str) +
    rfmtable['M_Score'].astype(str)
)

In [19]:
rfmtable.head()

,UserId,F_Value,R_Value,M_Value,R_Score,F_Score,M_Score,rfm_score
0,259266,1,2626,320608.80,1,1,5,115
1,259287,7,2303,17565.78,5,5,5,555
2,259308,4,2376,6954.48,2,4,4,244
3,259329,1,2319,7211.94,4,1,4,414
4,259350,1,2611,1387.20,1,1,2,112
